In [34]:
%pip install openai pydantic jinja2 python_dotenv ipywidgets d20

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [35]:
import dis
import os
import re
import tomllib
import functools

from jinja2 import Template
from numpy import diff
from pydantic import BaseModel
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv
from d20 import roll

import ipywidgets as widgets
from IPython.display import display, clear_output

from typing import Literal, Union

from sympy import fu

load_dotenv()

with open("data/example.toml", "rb") as f:
    config = tomllib.load(f)

message_pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
class Message(BaseModel):
    # Message from chatbot
    role: Literal["system", "assistant", "user"]
    content: str   

    # parse the content into dictionary
    def dict(self) -> dict:
        current_key = None
        parsed = {}
        lines = self.content.splitlines()
        # parse the content line by line
        for line in lines:
            res = message_pattern.search(line)
            if res is not None:
                current_key = res.group(1)
                if res.group(2) is not None:
                    parsed[current_key] = res.group(2)
                else:
                    parsed[current_key] = ""
            elif current_key is not None:
                parsed[current_key] += "\n" + line
        return parsed

class Chapter(BaseModel):
    id: str
    next: str|None = None
    max_turns: int
    backgrounds: list[str]
    rules: list[str]
    initial: str
    
    def __init__(self, **kargs):
        super().__init__(**kargs)
    
    def get_messages(self) -> list[Message]:
        rendered_rules = render_rules(self.rules)
        system_content = Template(system).render({"rules": rendered_rules, "backgrounds":self.backgrounds, "character": character})
        initial_content = Template(self.initial).render({"rules": self.rules, "character": character})
        return [
            Message(role="system", content=system_content),
            Message(role="assistant", content=initial_content)
        ]

version = config["version"]
system = config["system"]
character = config["character"]
chapters = config["chapters"]

messages:list[Message] =[]

tags:list[str] = ['r1']
items:list[dict] = character["items"]

print(f"Version: {version}, Chapters: {len(chapters)}")

def getChapterById(id:str) -> Chapter:
    for r in chapters:
        if r["id"] == id:
            return Chapter(**r) 
    return None

def choose_next_chapter():
    global current_chapter
    if current_chapter.next is not None:
        next = Template(current_chapter.next).render({"items": items, "tags": tags})
        print("Next round: ", next)
        next_chapter = getChapterById(next)
        if next_chapter is not None:
            current_chapter = next_chapter
            messages.clear()
            do_chat(None, lambda x: print(x, end=""))
        else:
            print(f"Chapter '{next}' not found.")
    else:
        print("No next chapter found.")


def count_messages(messages:list[Message], role:str) -> int:
    return len([msg for msg in messages if msg.role == role])

def chat_with_azure(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = AzureOpenAI(api_key=os.environ.get("AZURE_OPENAI_API_KEY"), 
                         azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
                         api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
                         )
    stream = client.chat.completions.create(
        model=os.environ.get("AZURE_OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        max_tokens=4000,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if len(chunk.choices) > 0 and chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)
    
def chat(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        max_tokens=4000,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)

def render_rules(rules:list[str]) -> str:
    lists = []
    for rule in rules:
        res = Template(rule).render({"character": character, "tags":tags, "items": items})
        if res is not None and res != "":
            lists.append(res)
    return lists

def do_chat(content:str|None, on_message: Union[callable, None] = None):
    if content is not None:
        turn = count_messages(messages, 'user') + 1
        if turn >= current_chapter.max_turns:
            content += f"\nwhisper: MAX_TURNS is reached."
        messages.append(Message(role="user", content=content))
        print(content)

    # change it to chat_with_azure to use Azure OpenAI
    concat = current_chapter.get_messages() + messages
    # for msg in concat:
    #     print(f"{msg.role}: {msg.content}")

    msg = chat(concat, on_message)

    # print all the messages:
    show_controls(msg)
    messages.append(msg)
        
# handle player's action
def on_action(b, index:int):
    user_msg = f"I select '{index}'."
    do_chat(user_msg, lambda x: print(x, end=""))

# handle player's skill check
def on_skill(b, skill:str, difficulty:str):
    roll_num = roll("1d100").total
    if roll_num < 5:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL SUCCESS!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return
    if roll_num >= 95:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL FAILURE!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return

    difficulty_num = 15 if skill not in character["skills"] else character["skills"][skill]
    if difficulty == "hard":
        difficulty_num = int(difficulty_num * 0.5)
    if difficulty == "extreme":
        difficulty_num = int(difficulty_num * 0.2)
    
    print(f"rolled {roll_num} against {difficulty_num} on skill '{skill}'...")
    
    if roll_num <= difficulty_num:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a success!"
    else:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a failure."

    do_chat(user_msg, lambda x: print(x, end=""))

def on_continue(b):
    user_msg = f"Continue..."
    do_chat(user_msg, lambda x: print(x, end=""))

def on_next(b):
    choose_next_chapter()
        
def on_retry(b):
    # delete last assistant message
    messages.pop()
    do_chat(None, lambda x: print(x, end=""))

def on_item(b, item:dict):
    user_msg = Template(item["description"]).render({"character": character, "tags":tags, "items": items})
    user_msg = f"I use '{item['id']}' - {user_msg}"
    do_chat(user_msg, lambda x: print(x, end=""))

action_pattern = re.compile(r"(\d+). *(.+)")
def show_controls(msg:Message):
    dict = msg.dict()
    buttons = []

    if "choices" in dict.keys():
        matches = action_pattern.finditer(dict["choices"])
        for match in matches:
            btn = widgets.Button(description=match.group(0), layout=widgets.Layout(width="fit-content"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        btn = widgets.Button(description=f"{dict['skill']} - {dict['difficulty']}", layout=widgets.Layout(width="fit-content"))
        btn.on_click(functools.partial(on_skill, skill=dict['skill'], difficulty=dict['difficulty']))
        buttons.append(btn)
    elif "ending" in dict.keys():
        btn = widgets.Button(description=f"NEXT", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_next)
        buttons.append(btn)
    elif "game_over" in dict.keys():
        print("\nGame Over - Hit 'Run All' button to restart again.")
    else:
        btn = widgets.Button(description="Continue", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_continue)
        buttons.append(btn)
    
    if "add_tag" in dict.keys() and dict["add_tag"] not in tags:
        tags.append(dict["add_tag"])

    if "remove_tag" in dict.keys() and dict["remove_tag"] in tags:
        tags.remove(dict["remove_tag"])
    
    if "add_item" in dict.keys() and dict["add_item"] not in items:
        items.append(dict["add_item"])
    
    if "remove_item" in dict.keys() and dict["remove_item"] in items:
        items.remove(dict["remove_item"])
    
    controls = []
    if len(messages) > 0:
        retry_btn = widgets.Button(description="Retry", layout=widgets.Layout(width="fit-content"))
        retry_btn.on_click(on_retry)
        controls.append(retry_btn)

    if len(tags) > 0:
        tags_lbl = widgets.Label(value=f"Tags: {', '.join(tags)}")
        controls.append(tags_lbl)

    inventory = [widgets.Label(value="Inventory:")]
    if len(items) > 0:
        for item in items:
            item_btn = widgets.Button(description=item["id"], layout=widgets.Layout(width="fit-content"))
            item_btn.on_click(functools.partial(on_item, item=item))
            inventory.append(item_btn)
        controls.append(widgets.HBox(inventory))
    
    display(widgets.VBox(buttons))

    if (len(controls) > 0):
        display(widgets.HBox(controls))

current_chapter = getChapterById("r1")

# locals_dict = {}
# s = "'r1' in tags"
# source = f"def f(tags): return {s}"
# exec(source, {}, locals_dict)
# print(locals_dict['f'](['r2']))

Version: 0.0.3, Chapters: 1


In [36]:
initials = current_chapter.get_messages()
print(initials[1].content)
show_controls(initials[1])

background: house_front
narration: 
You, John Doe, a retired detective, who has lost his family in a tragic accident, and using alcohol to cope with the loss.

You have been investigating a series of murders in the town for many years.
The recent clues led you to a abandoned house in the forest. Now you are standing in front of the house.
choices:
1. Enter the house from the front door.
2. Look around the house to find some clues.
3. Drink the alcohol to gain some courage.


I select '1'.
background: house_dining_room
narration: 
You enter the house, the first room you find is a dining room. It's old and dusty, with a large wooden table in the middle. You hear strange noises from the floor above.
choices:
1. Investigate the noises upstairs.
2. Search the dining room for clues.
3. Proceed to the next room.

I use 'lighter' - A lighter
narration: 
You take out your old lighter, the flame flickers in the dark room, casting eerie shadows on the walls. The light helps you to check the surroundings better.
choices:
1. Investigate the noises upstairs.
2. Search the dining room for clues.
3. Proceed to the next room.

I select '3'.
background: house_meeting_room
narration: 
You move to the next room, a meeting room. You see a large wooden bookshelf covering a part of the wall. As you scan the room, you notice something unusual about the bookshelf.
choices:
1. Investigate the bookshelf.
2. Ignore it and move to the next room.
3. Go upstairs to investigate the noise.

I use 'flashlight' - A flashlight with low battery
narration: 
You take out your flashlight, despite its low battery, it casts a stronger light than your lighter. You point it at the bookshelf, the light revealing a hidden door behind the books.
choices:
1. Try to open the hidden door.
2. Ignore it and move to the next room.
3. Go upstairs to investigate the noise.